In [6]:
import json
import os

import argparse
import torch
import yaml
from helper import *
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from efficientdet_arch.backbone import EfficientDetBackbone
from efficientdet_arch.efficientdet.utils import BBoxTransform, ClipBoxes
from efficientdet_arch.utils.utils import preprocess, invert_affine, postprocess, boolean_string

In [7]:
INPUT_DIM = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]

PROJECT_NAME = "goodbadchili"
CLASSES = ["good_chili", "bad_chili"]
EFFICIENTNET_COMPOUND_COEF = 0
WEIGHT_PATH = "d0-chili-12E.pth"
ANCHOR_RATIOS=[(1.0, 0.7), (1.0, 1.0), (1.0, 1.5)]
ANCHOR_SCALES=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]

THRESHOLD=0.05
NMS_THRESHOLD=0.5

In [ ]:
PROJECT_DIR = os.path.join(os.path.abspath(os.getcwd()), "datasets", PROJECT_NAME, "COCO")
RESULT_FNAME = f"{WEIGHT_PATH[:-4]}_results.json"
ANNOTATION_PATH = os.path.join(PROJECT_DIR, "annotations", "instances_test.json")
IMAGE_DIR = os.path.join(PROJECT_DIR, "test")
COCO_INFO = COCO(ANNOTATION_PATH)
IMAGE_IDS = COCO_INFO.getImgIds()[:10000]

In [ ]:
model = EfficientDetBackbone(num_classes=len(CLASSES), compound_coef=EFFICIENTNET_COMPOUND_COEF, ratios=ANCHOR_RATIOS, scales=ANCHOR_SCALES)
model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))
model.requires_grad_(False)
model.eval()
model.cuda("cuda:0")

In [ ]:
results = []
regressBoxes, clipBoxes = BBoxTransform(), ClipBoxes()
for image_id in tqdm(IMAGE_IDS):
    image_info = COCO_INFO.loadImgs(image_id)[0]
    image_path = os.path.join(IMAGE_DIR, image_info['file_name'])
    _, framed_imgs, framed_metas = preprocess(image_path, max_size=INPUT_DIM[EFFICIENTNET_COMPOUND_COEF], mean=CONFIG["mean"], std=CONFIG["std"])
    x = torch.from_numpy(framed_imgs[0]).cuda("cuda:0").float().unsqueeze(0).permute(0, 3, 1, 2)
    _, regression, classification, anchors = model(x)
    preds = postprocess(x, anchors, regression, classification, regressBoxes, clipBoxes, threshold, nms_threshold)
    if not preds: continue
    preds = invert_affine(framed_metas, preds)[0]
    scores, class_ids, rois = preds['scores'], preds['class_ids'], preds['rois']
    if rois.shape[0] > 0:   # x1,y1,x2,y2 -> x1,y1,w,h
        rois[:, 2] -= rois[:, 0]
        rois[:, 3] -= rois[:, 1]
        bbox_score = scores
        for roi_id in range(rois.shape[0]):
            score, label, box = float(bbox_score[roi_id]), int(class_ids[roi_id]), rois[roi_id, :]
            image_result = {
                'image_id': image_id,
                'category_id': label + 1,
                'score': float(score),
                'bbox': box.tolist(),
            }
            results.append(image_result)
            
if len(results) == 0:
    raise Exception('Model does not produce any valid output, check model input')

with open(os.path.join(os.path.abspath(os.getcwd()), RESULT_FNAME) as f:
    json.dump(results, f, 'w'), indent=4)

In [ ]:
coco_eval = COCOeval(COCO_INFO, COCO_INFO.loadRes(os.path.join(os.path.abspath(os.getcwd()), RESULT_FNAME)), 'bbox')
coco_eval.params.imgIds = IMAGE_IDS
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()